In [0]:
import pandas as pd
import numpy as np 
import xgboost as xgb
from sklearn.metrics import roc_auc_score 
import re
import traceback
import string
import datetime
from itertools import chain 

class ks_search(): 
    def __init__ (self , num_iter , p3 , dev1 , bad_dev , weight_bad , oot1 , bad_oot , weight_oot):
        self.num_iter = num_iter 
        self.p3 = p3 
        self.dev1 = dev1 
        self.bad_dev = bad_dev
        self.weight_bad = weight_bad 
        self.oot1 = oot1 
        self.bad_oot = bad_oot 
        self.weight_oot = weight_oot 
        self.result_out = None 
        
    
    def randomsearch_KS(self): 
        output_full = []
        xg_cust = xgb.XGBClassifier()
        mks = 100
        m10 = 100
        m20 = 100
        num_iter = self.num_iter
        for j in range(num_iter):
            p4 = {}
            print("Iteration number : " + str(j))
            p5 = eval(self.p3)
            for i in ['colsample_bylevel','colsample_bytree','learning_rate','max_depth','min_child_weight','n_estimators','subsample'] :
                try: 
                    p4[i] = p5[i] 
                except : 
                    print(i + ' Unavailable') 
            xg_cust.set_params(**p4)
            xg_cust.fit(self.dev1,self.bad_dev , sample_weight = self.weight_bad , eval_set = [(self.oot1 , self.bad_oot , self.weight_oot)] ,
                  eval_metric = 'auc' ,early_stopping_rounds = 8)
            pred_dev = xg_cust.predict_proba(self.dev1 , ntree_limit = xg_cust.best_ntree_limit)[:,1]
            pred_oot = xg_cust.predict_proba(self.oot1, ntree_limit = xg_cust.best_ntree_limit)[:,1]
            res1 = self.result_ks(self.bad_dev , pred_dev,self.bad_oot , pred_oot  ,self.weight_bad, self.weight_oot)
            r2 = res1[2]
            r2['Diff'] = r2['Dev'] -  r2['OOT(tst)']
            ks = r2['Diff'].iloc[2]
            n10 = r2['Diff'].iloc[3]
            n20 = r2['Diff'].iloc[4]    
            bdrate_chk =  res1[1]['bad_rate'].str.replace('%','').astype('float').is_monotonic_decreasing &  res1[0]['bad_rate'].str.replace('%','').astype('float').is_monotonic_decreasing
            if(ks < mks and n10 < m10 and n20 < m20 and bdrate_chk == True):
                mks = ks
                m10 = n10
                m20 = n20
                promod = xg_cust
            dout = r2
            dout['Bad_Rate'] = bdrate_chk
            dout['min_prob'] = pred_oot.min()
            dout['max_prob'] = pred_oot.max()
            dout['predicted_bad_rate'] = pred_oot.mean()
            dout['param_dict'] = str(p4)
            dout['Iteration'] = j
            dout['Best_N_Tree']  = xg_cust.best_ntree_limit
            output_full.append(dout)
     
        self.result_out = output_full
    
    def KS_train(self,y_train,y_train_rf,wt,bins = 20 , mode = None):
        pred=pd.DataFrame(y_train)
        pred['Prob_bad']=y_train_rf
        data=pred
        data.rename(columns={'overall_bad': 'bad'}, inplace=True)
        data.columns = ['bad','Prob_bad']
        data['wt'] = wt
        data['good'] = 1 - data.bad
        if mode == 'score':
            data=(data.sort_values(by=(['Prob_bad','bad']),ascending=[True,False])).reset_index(drop=True)
        else :
            data=(data.sort_values(by=(['Prob_bad','bad']),ascending=[False,False])).reset_index(drop=True)
        data['wt_cumsum'] = data['wt'].cumsum()
        data['wt_good'] = data.apply(lambda x: 0 if x['bad'] == 1 else x['wt'],axis=1)
        data['wt_bad'] = data.apply(lambda x: 0 if x['good'] == 1 else x['wt'],axis=1)
        data['bucket'] = pd.cut(data.wt_cumsum, bins)
        grouped = data.groupby('bucket', as_index = False)
        agg1 = grouped.min().Prob_bad
        agg1 = pd.DataFrame(grouped.min().Prob_bad, columns = ['Decile'])
        agg1['Decile']  = pd.Series(list(range(5,105,int(100/bins))))
        agg1['min_scr']= grouped.min().Prob_bad
        agg1['max_scr'] = grouped.max().Prob_bad
        agg1['bads'] = grouped.sum().wt_bad
        agg1['goods'] = grouped.sum().wt_good
        agg1['total'] = agg1.bads + agg1.goods
        agg1['bad_rate'] = (agg1.bads / agg1.total).apply('{0:.2%}'.format)
        agg1['cumm_bad'] =(agg1.bads / data.wt_bad.sum()).cumsum()* 100
        agg1['cumm_good'] =(agg1.goods / data.wt_good.sum()).cumsum()* 100
        agg1['KS']=np.abs(np.round((agg1['cumm_bad']-agg1['cumm_good']),4)) 
        flag = lambda x: '<----' if x == agg1.KS.max() else ''
        agg1['max_ks'] = agg1.KS.apply(flag)
        auc = roc_auc_score(y_train,y_train_rf,sample_weight = wt )
        gini = 2*auc - 1
        agg1['gini'] = gini
        agg1['auc'] = auc
        Score_Perf_Dec=agg1[["min_scr","max_scr","bads","goods","total","bad_rate",'cumm_bad','cumm_good','gini','KS','auc','max_ks']]        
        return agg1
    
    def result_ks(self , x , y , x1 , y1 , wt1  , wt2 , bins = 10 , mode = None):
        a1 = self.KS_train(x ,  y , wt1  , bins , mode)
        a2 = self.KS_train(x1 , y1 , wt2 , bins , mode)
        res = pd.DataFrame({},index=list(range(0,5)),columns=['Dev','OOT(tst)'])
        res['Dev'][0] = a1.auc[0]
        res['Dev'][1] = a1.gini[0]
        res['Dev'][2] = a1.KS.max()
        res['Dev'][3] = a1.query('Decile == 5')['cumm_bad'].reset_index(drop = True)[0]
        res['Dev'][4] = a1.query('Decile == 15')['cumm_bad'].reset_index(drop = True)[0]
        res['OOT(tst)'][0] = a2.auc[0]
        res['OOT(tst)'][1] = a2.gini[0]
        res['OOT(tst)'][2] = a2.KS.max()
        res['OOT(tst)'][3] = a2.query('Decile == 5')['cumm_bad'].reset_index(drop = True)[0]
        res['OOT(tst)'][4] = a2.query('Decile == 15')['cumm_bad'].reset_index(drop = True)[0]
        res.index = [['ROC_AUC','Gini','KS','10%','20%']]
        return a1,a2,res